# 🧠 TRM-OCR: Recursive Reasoning for LaTeX OCR

A **simplified, one-click runnable** implementation of TRM for LaTeX OCR.

## Key Features
- **Parameter-Shared Recursive Reasoning**: Single module reused recursively
- **Adaptive Computation Time (ACT)**: Learn when to stop thinking
- **Character-level LaTeX Generation**: Image → LaTeX sequence

---
**Run all cells sequentially (Ctrl+F9 or Runtime → Run all)**

# CELL 1: Install & Import

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 1: Install & Import
# ═══════════════════════════════════════════════════════════════════════════════

!pip install -q torch einops tqdm numpy pillow pydantic wandb transformers datasets

import os, math, random
from typing import Tuple, Dict, List
from dataclasses import dataclass
from collections import Counter
from io import BytesIO

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler
from torch.utils.checkpoint import checkpoint

import numpy as np
from einops import rearrange
from tqdm.auto import tqdm
from PIL import Image
import torchvision.transforms as transforms
import wandb
from datasets import load_dataset

# Set seed
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"✅ PyTorch {torch.__version__} | Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

# CELL 2: Configuration

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 2: Configuration
# ═══════════════════════════════════════════════════════════════════════════════

CONFIG = {
    # Data
    "batch_size": 16,           # 减小以防 OOM
    "max_latex_len": 64,        # LaTeX 序列最大长度
    "img_size": 224,
    
    # Vision Encoder
    "patch_size": 16,
    "vision_embed_dim": 256,    # 缩小以节省显存
    "vision_depth": 4,
    "vision_heads": 4,
    
    # TRM Core
    "hidden_size": 256,
    "expansion": 4.0,
    "num_heads": 4,
    
    # Recursion (key params!)
    "H_cycles": 2,
    "L_cycles": 4,
    "reasoning_layers": 2,
    
    # ACT
    "halt_max_steps": 6,
    "halt_exploration_prob": 0.15,
    "halt_threshold": 0.9,  # 新增：准确率阈值用于halt
    
    # Training
    "num_epochs": 10,
    "learning_rate": 5e-4,
    "weight_decay": 0.05,
    "warmup_ratio": 0.1,
    "use_amp": True,
    
    # Data size
    "train_size": 10000,
    "val_size": 1000,
    
    # WANDB
    "wandb_project": "TRM-LaTeX-OCR",
    "wandb_api_key": "",  # 你的 key
}

# Validate
assert CONFIG["vision_embed_dim"] % CONFIG["vision_heads"] == 0
assert CONFIG["hidden_size"] % CONFIG["num_heads"] == 0

total_reasoning = CONFIG["H_cycles"] * CONFIG["L_cycles"] + CONFIG["H_cycles"]
print(f"✅ Config validated")
print(f"   Reasoning passes per ACT step: {total_reasoning}")
print(f"   Max total passes: {total_reasoning * CONFIG['halt_max_steps']}")

# CELL 3: Base Components 

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 3: Base Components (Attention, MLP, Vision Encoder)
# ═══════════════════════════════════════════════════════════════════════════════

def trunc_normal_init_(tensor, std=1.0):
    with torch.no_grad():
        tensor.normal_(0, std).clamp_(-2*std, 2*std)
    return tensor

def rms_norm(x, eps=1e-5):
    return x * torch.rsqrt(x.float().square().mean(-1, keepdim=True) + eps).to(x.dtype)

def rotate_half(x):
    x1, x2 = x[..., :x.shape[-1]//2], x[..., x.shape[-1]//2:]
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary_pos_emb(q, k, cos, sin):
    q_embed = (q * cos.unsqueeze(-2)) + (rotate_half(q) * sin.unsqueeze(-2))
    k_embed = (k * cos.unsqueeze(-2)) + (rotate_half(k) * sin.unsqueeze(-2))
    return q_embed, k_embed

class CastedLinear(nn.Module):
    def __init__(self, in_f, out_f, bias=True):
        super().__init__()
        self.weight = nn.Parameter(trunc_normal_init_(torch.empty(out_f, in_f), std=1/(in_f**0.5)))
        self.bias = nn.Parameter(torch.zeros(out_f)) if bias else None
    def forward(self, x):
        return F.linear(x, self.weight.to(x.dtype), self.bias.to(x.dtype) if self.bias is not None else None)

class CastedEmbedding(nn.Module):
    def __init__(self, num, dim, std):
        super().__init__()
        self.weight = nn.Parameter(trunc_normal_init_(torch.empty(num, dim), std=std))
    def forward(self, x):
        return F.embedding(x, self.weight)

class RotaryEmbedding(nn.Module):
    def __init__(self, dim, max_pos, base=10000.0):
        super().__init__()
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
        t = torch.arange(max_pos).float()
        freqs = torch.outer(t, inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer('cos', emb.cos(), persistent=False)
        self.register_buffer('sin', emb.sin(), persistent=False)
    def forward(self):
        return self.cos, self.sin

class Attention(nn.Module):
    def __init__(self, hidden, head_dim, n_heads, n_kv_heads):
        super().__init__()
        self.n_heads, self.n_kv_heads, self.head_dim = n_heads, n_kv_heads, head_dim
        self.qkv = CastedLinear(hidden, (n_heads + 2*n_kv_heads)*head_dim, bias=False)
        self.out = CastedLinear(head_dim*n_heads, hidden, bias=False)
    def forward(self, cos_sin, x):
        B, S, _ = x.shape
        qkv = self.qkv(x).view(B, S, self.n_heads + 2*self.n_kv_heads, self.head_dim)
        q, k, v = qkv[:,:,:self.n_heads], qkv[:,:,self.n_heads:self.n_heads+self.n_kv_heads], qkv[:,:,self.n_heads+self.n_kv_heads:]
        if cos_sin:
            cos, sin = cos_sin
            q, k = apply_rotary_pos_emb(q, k, cos[:S], sin[:S])
        q, k, v = [rearrange(t, 'B S H D -> B H S D') for t in [q, k, v]]
        out = F.scaled_dot_product_attention(q, k, v)
        return self.out(rearrange(out, 'B H S D -> B S (H D)'))

class SwiGLU(nn.Module):
    def __init__(self, hidden, exp):
        super().__init__()
        inter = ((int(exp * hidden * 2 / 3) + 255) // 256) * 256
        self.gate_up = CastedLinear(hidden, inter*2, bias=False)
        self.down = CastedLinear(inter, hidden, bias=False)
    def forward(self, x):
        g, u = self.gate_up(x).chunk(2, dim=-1)
        return self.down(F.silu(g) * u)

class ViTBlock(nn.Module):
    def __init__(self, dim, heads):
        super().__init__()
        self.norm1, self.norm2 = nn.LayerNorm(dim), nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(dim, heads, batch_first=True)
        self.mlp = nn.Sequential(nn.Linear(dim, dim*4), nn.GELU(), nn.Linear(dim*4, dim))
    def forward(self, x):
        x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        return x + self.mlp(self.norm2(x))

class VisionEncoder(nn.Module):
    def __init__(self, img_size, patch_size, dim, depth, heads):
        super().__init__()
        self.n_patches = (img_size // patch_size) ** 2
        self.proj = nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size)
        self.cls = nn.Parameter(torch.randn(1, 1, dim) * 0.02)
        self.pos = nn.Parameter(torch.randn(1, self.n_patches + 1, dim) * 0.02)
        self.blocks = nn.ModuleList([ViTBlock(dim, heads) for _ in range(depth)])
        self.norm = nn.LayerNorm(dim)
        self.dim = dim
    def forward(self, x):
        B = x.shape[0]
        x = self.proj(x).flatten(2).transpose(1, 2)
        x = torch.cat([self.cls.expand(B, -1, -1), x], dim=1) + self.pos
        for blk in self.blocks:
            x = blk(x)
        return self.norm(x)

print("✅ Base components defined")

# CELL 4: TRM-OCR Model with Autoregressive Decoder

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 4: TRM-OCR Model with Autoregressive Decoder (Fixed)
# ═══════════════════════════════════════════════════════════════════════════════

@dataclass
class TRMCarry:
    z_H: torch.Tensor
    z_L: torch.Tensor

@dataclass
class ACTCarry:
    inner: TRMCarry
    steps: torch.Tensor
    halted: torch.Tensor
    data: Dict[str, torch.Tensor]

class TRMBlock(nn.Module):
    def __init__(self, hidden, heads, exp, eps=1e-5):
        super().__init__()
        self.attn = Attention(hidden, hidden//heads, heads, heads)
        self.mlp = SwiGLU(hidden, exp)
        self.eps = eps
    def forward(self, cos_sin, x):
        x = rms_norm(x + self.attn(cos_sin, x), self.eps)
        return rms_norm(x + self.mlp(x), self.eps)

# ⭐ 修复的 DecoderLayer
class DecoderLayer(nn.Module):
    """单层Transformer Decoder with cross-attention"""
    def __init__(self, hidden, heads):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(hidden, heads, batch_first=True)
        self.cross_attn = nn.MultiheadAttention(hidden, heads, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(hidden, hidden * 4),
            nn.GELU(),
            nn.Linear(hidden * 4, hidden)
        )
        self.norm1 = nn.LayerNorm(hidden)
        self.norm2 = nn.LayerNorm(hidden)
        self.norm3 = nn.LayerNorm(hidden)
    
    def forward(self, x, memory, tgt_mask=None):
        # ⭐ 如果没有提供mask，生成causal mask
        if tgt_mask is None:
            seq_len = x.shape[1]
            tgt_mask = torch.triu(
                torch.ones(seq_len, seq_len, device=x.device, dtype=torch.bool),
                diagonal=1
            )
        
        # Self-attention (causal)
        x = x + self.self_attn(
            self.norm1(x), self.norm1(x), self.norm1(x),
            attn_mask=tgt_mask,  # ⭐ 只传mask，不用is_causal
            need_weights=False
        )[0]
        
        # Cross-attention to vision features
        x = x + self.cross_attn(
            self.norm2(x), memory, memory,
            need_weights=False
        )[0]
        
        # MLP
        x = x + self.mlp(self.norm3(x))
        return x

class TRMOCR(nn.Module):
    """TRM for LaTeX OCR with Autoregressive Decoder"""
    def __init__(self, cfg, vocab_size, use_checkpoint=True):
        super().__init__()
        self.cfg = cfg
        self.use_checkpoint = use_checkpoint
        self.vocab_size = vocab_size
        self.max_len = cfg["max_latex_len"]
        
        # Vision Encoder
        self.vision = VisionEncoder(cfg["img_size"], cfg["patch_size"], 
                                    cfg["vision_embed_dim"], cfg["vision_depth"], cfg["vision_heads"])
        self.n_vis = self.vision.n_patches + 1
        
        # Project vision to hidden
        self.vis_proj = CastedLinear(cfg["vision_embed_dim"], cfg["hidden_size"], bias=False)
        
        # Sequence length = vision tokens only
        self.seq_len = self.n_vis
        self.rope = RotaryEmbedding(cfg["hidden_size"]//cfg["num_heads"], self.seq_len)
        self.scale = math.sqrt(cfg["hidden_size"])
        
        # TRM reasoning module (reused!)
        self.reason = nn.ModuleList([TRMBlock(cfg["hidden_size"], cfg["num_heads"], cfg["expansion"]) 
                                     for _ in range(cfg["reasoning_layers"])])
        
        # 自回归Decoder
        self.token_embed = nn.Embedding(vocab_size, cfg["hidden_size"])
        self.pos_embed = nn.Embedding(self.max_len, cfg["hidden_size"])
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(cfg["hidden_size"], cfg["num_heads"]) 
            for _ in range(2)  # 2层decoder
        ])
        self.output_proj = nn.Linear(cfg["hidden_size"], vocab_size)
        
        # Halt decision head
        self.q_head = CastedLinear(cfg["hidden_size"], 2, bias=True)
        
        # Init states
        self.H_init = nn.Parameter(trunc_normal_init_(torch.empty(cfg["hidden_size"]), std=1))
        self.L_init = nn.Parameter(trunc_normal_init_(torch.empty(cfg["hidden_size"]), std=1))
        
        with torch.no_grad():
            self.q_head.weight.zero_()
            self.q_head.bias.fill_(-2)
        
        # ⭐ 预先创建causal mask并注册为buffer（避免重复创建）
        causal_mask = torch.triu(
            torch.ones(self.max_len, self.max_len, dtype=torch.bool),
            diagonal=1
        )
        self.register_buffer('causal_mask', causal_mask, persistent=False)
    
    def _encode(self, img):
        vis = self.vis_proj(self.vision(img))
        return self.scale * vis
    
    def _reason_step(self, z_L, z_H, inp, cos_sin):
        for _ in range(self.cfg["L_cycles"]):
            z_L = z_L + z_H + inp
            for layer in self.reason:
                z_L = layer(cos_sin, z_L)
        z_H = z_H + z_L
        for layer in self.reason:
            z_H = layer(cos_sin, z_H)
        return z_L, z_H
    
    def empty_carry(self, B, dev):
        z = torch.zeros(B, self.seq_len, self.cfg["hidden_size"], device=dev)
        return TRMCarry(z_H=z.clone(), z_L=z.clone())
    
    def reset_carry(self, flag, carry):
        B, S, D = carry.z_H.shape
        H = self.H_init.view(1,1,D).expand(B,S,D)
        L = self.L_init.view(1,1,D).expand(B,S,D)
        mask = flag.view(-1,1,1)
        return TRMCarry(z_H=torch.where(mask, H, carry.z_H), z_L=torch.where(mask, L, carry.z_L))
    
    def initial_carry(self, batch):
        B = batch["image"].shape[0]
        dev = batch["image"].device
        return ACTCarry(
            inner=self.empty_carry(B, dev),
            steps=torch.zeros(B, dtype=torch.int32, device=dev),
            halted=torch.ones(B, dtype=torch.bool, device=dev),
            data={k: torch.empty_like(v) for k,v in batch.items() if isinstance(v, torch.Tensor)}
        )
    
    def decode_autoregressive(self, memory, target=None, max_len=None):
        """
        自回归解码
        Args:
            memory: vision features [B, seq_len, hidden]
            target: ground truth tokens [B, max_len] (训练时使用)
            max_len: 最大生成长度（推理时使用）
        Returns:
            logits: [B, max_len, vocab_size]
        """
        B = memory.shape[0]
        device = memory.device
        
        if target is not None:
            # 训练模式：Teacher Forcing
            tgt_len = target.shape[1]
            
            # Token + Position embedding
            tok_emb = self.token_embed(target)
            pos_ids = torch.arange(tgt_len, device=device).unsqueeze(0).expand(B, -1)
            pos_emb = self.pos_embed(pos_ids)
            tgt = tok_emb + pos_emb
            
            # ⭐ 使用预先创建的causal mask
            mask = self.causal_mask[:tgt_len, :tgt_len]
            
            # Decoder layers
            for layer in self.decoder_layers:
                tgt = layer(tgt, memory, tgt_mask=mask)
            
            # Output projection
            logits = self.output_proj(tgt)
            return logits
        
        else:
            # 推理模式：自回归生成
            max_len = max_len or self.max_len
            generated = torch.full((B, 1), 1, dtype=torch.long, device=device)  # SOS=1
            
            for i in range(max_len - 1):
                # Embed current sequence
                tok_emb = self.token_embed(generated)
                pos_ids = torch.arange(generated.shape[1], device=device).unsqueeze(0).expand(B, -1)
                pos_emb = self.pos_embed(pos_ids)
                tgt = tok_emb + pos_emb
                
                # ⭐ 使用对应长度的causal mask
                curr_len = generated.shape[1]
                mask = self.causal_mask[:curr_len, :curr_len]
                
                # Decode
                for layer in self.decoder_layers:
                    tgt = layer(tgt, memory, tgt_mask=mask)
                
                # 只取最后一个token的预测
                logits = self.output_proj(tgt[:, -1:])
                next_token = logits.argmax(-1)
                
                # 拼接
                generated = torch.cat([generated, next_token], dim=1)
            
            # 返回完整序列的logits
            tok_emb = self.token_embed(generated)
            pos_ids = torch.arange(generated.shape[1], device=device).unsqueeze(0).expand(B, -1)
            pos_emb = self.pos_embed(pos_ids)
            tgt = tok_emb + pos_emb
            
            mask = self.causal_mask[:generated.shape[1], :generated.shape[1]]
            for layer in self.decoder_layers:
                tgt = layer(tgt, memory, tgt_mask=mask)
            
            logits = self.output_proj(tgt)
            return logits
    
    def forward(self, carry, batch):
        inner = self.reset_carry(carry.halted, carry.inner)
        steps = torch.where(carry.halted, torch.zeros_like(carry.steps), carry.steps)
        
        # Update data
        data = {}
        for k, v in carry.data.items():
            if k in batch:
                exp = (1,) * (batch[k].ndim - 1)
                data[k] = torch.where(carry.halted.view(-1, *exp), batch[k], v)
            else:
                data[k] = v
        
        # Encode vision features
        cos_sin = self.rope()

        #inp = self._encode(data["image"])

        # 提升 1 [视觉编码缓存]
        if "vis_feat" not in data:
        data["vis_feat"] = self._encode(data["image"])
        inp = data["vis_feat"]

        z_H, z_L = inner.z_H, inner.z_L
        
        # TRM Recursive reasoning
        for _ in range(self.cfg["H_cycles"]):
            if self.use_checkpoint and self.training:
                z_L, z_H = checkpoint(self._reason_step, z_L, z_H, inp, cos_sin, use_reentrant=False)
            else:
                z_L, z_H = self._reason_step(z_L, z_H, inp, cos_sin)
        
        # 自回归解码
        if self.training:
            seq_logits = self.decode_autoregressive(z_H, target=data["target"])
        else:
            seq_logits = self.decode_autoregressive(z_H, target=None, max_len=self.max_len)
        
        # Halt decision
        cls = z_H[:, 0]
        q_logits = self.q_head(cls).float()
        
        # ACT halting logic
        with torch.no_grad():
            steps = steps + 1
            halted = steps >= self.cfg["halt_max_steps"]
            if self.training and self.cfg["halt_max_steps"] > 1:
                halt_prob = torch.sigmoid(q_logits[:, 0])
                halted = halted | (halt_prob > 0.5)
                min_steps = (torch.rand_like(halt_prob) < self.cfg["halt_exploration_prob"]) * \
                           torch.randint_like(steps, 2, self.cfg["halt_max_steps"]+1)
                halted = halted & (steps >= min_steps)
        
        new_inner = TRMCarry(z_H=z_H.detach(), z_L=z_L.detach())
        new_carry = ACTCarry(inner=new_inner, steps=steps, halted=halted, data=data)
        return new_carry, {"seq": seq_logits, "q_halt": q_logits[:, 0]}

print("✅ TRM-OCR model with Autoregressive Decoder defined")

# CELL 5: Dataset & DataLoader (LaTeX OCR)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 5: Dataset & DataLoader (LaTeX OCR)
# ═══════════════════════════════════════════════════════════════════════════════

class LaTeXTokenizer:
    """Simple character-level tokenizer for LaTeX"""
    def __init__(self):
        self.PAD = 0
        self.SOS = 1
        self.EOS = 2
        self.UNK = 3
        self.c2i = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
        self.i2c = {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '<unk>'}
    
    def fit(self, texts):
        chars = set()
        for text in tqdm(texts, desc="Building vocab"):
            chars.update(text)
        for i, c in enumerate(sorted(chars), start=4):
            self.c2i[c] = i
            self.i2c[i] = c
        print(f"✅ LaTeX Tokenizer: {len(self.c2i)} tokens")
    
    def encode(self, text, max_len):
        """Encode text to token ids with SOS and EOS"""
        tokens = [self.SOS]
        for c in text[:max_len-2]:  # Reserve space for SOS and EOS
            tokens.append(self.c2i.get(c, self.UNK))
        tokens.append(self.EOS)
        # Pad to max_len
        tokens = tokens + [self.PAD] * (max_len - len(tokens))
        return torch.tensor(tokens[:max_len], dtype=torch.long)
    
    def decode(self, ids):
        """Decode token ids back to text"""
        chars = []
        for i in ids:
            i = i.item() if isinstance(i, torch.Tensor) else i
            if i == self.EOS:
                break
            if i not in (self.PAD, self.SOS):
                chars.append(self.i2c.get(i, '?'))
        return ''.join(chars)
    
    def __len__(self):
        return len(self.c2i)

class LaTeXOCRDataset(Dataset):
    """Dataset for LaTeX OCR"""
    def __init__(self, data, tokenizer, img_size=224, max_len=64):
        self.data = data
        self.tok = tokenizer
        self.max_len = max_len
        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.error_count = 0  # 追踪错误数量
    
    def __len__(self): 
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        # Image
        try:
            img = item['image']
            if not isinstance(img, Image.Image):
                img = Image.open(BytesIO(img['bytes'])) if isinstance(img, dict) else img
            img = self.transform(img.convert('RGB'))
        except Exception as e:
            # ⭐ 添加错误日志
            if self.error_count < 5:  # 只打印前5个错误
                print(f"⚠️ Image load failed at index {idx}: {e}")
                self.error_count += 1
            img = torch.zeros(3, 224, 224)
        
        # LaTeX text
        text = item.get('text', '') or item.get('formula', '') or ''
        tokens = self.tok.encode(text, self.max_len)
        
        return {"image": img, "target": tokens}

# ═══════════════════════════════════════════════════════════════════════════════
# Load LaTeX OCR Data
# ═══════════════════════════════════════════════════════════════════════════════
print("📦 Loading LaTeX OCR data...")

try:
    # Load dataset
    dataset = load_dataset("linxy/LaTeX_OCR", split="train")
    print(f"✅ Loaded {len(dataset)} samples")
    
    # Split into train/val
    train_size = min(CONFIG["train_size"], int(len(dataset) * 0.9))
    val_size = min(CONFIG["val_size"], len(dataset) - train_size)
    
    train_data = dataset.select(range(train_size))
    val_data = dataset.select(range(train_size, train_size + val_size))
    
    print(f"   Train: {len(train_data)} | Val: {len(val_data)}")
    
    # Show sample
    print(f"\n📋 Sample:")
    sample = train_data[0]
    text_key = 'text' if 'text' in sample else 'formula'
    print(f"   LaTeX: {sample[text_key][:80]}...")
    
except Exception as e:
    print(f"⚠️ LaTeX_OCR unavailable: {e}")
    print("   Creating synthetic data...")
    
    class SyntheticLaTeX:
        def __init__(self, n):
            self.n = n
            self.formulas = [
                r"x^2 + y^2 = z^2",
                r"\frac{a}{b}",
                r"\sum_{i=1}^{n} x_i",
                r"\int_0^1 f(x) dx",
                r"e^{i\pi} + 1 = 0",
                r"\sqrt{a^2 + b^2}",
                r"\alpha + \beta = \gamma",
                r"\lim_{x \to 0} \frac{\sin x}{x}",
            ]
        def __len__(self): return self.n
        def __getitem__(self, i):
            return {'text': self.formulas[i % len(self.formulas)],
                    'image': Image.new('RGB', (224, 224), (255, 255, 255))}
        def select(self, indices):
            return [self[i] for i in indices]
    
    full_data = SyntheticLaTeX(11000)
    train_data = full_data.select(range(CONFIG["train_size"]))
    val_data = full_data.select(range(CONFIG["train_size"], CONFIG["train_size"] + CONFIG["val_size"]))

# ═══════════════════════════════════════════════════════════════════════════════
# Build Tokenizer
# ═══════════════════════════════════════════════════════════════════════════════
tokenizer = LaTeXTokenizer()
text_key = 'text' if 'text' in train_data[0] else 'formula'
tokenizer.fit([item[text_key] for item in train_data if text_key in item])

# ═══════════════════════════════════════════════════════════════════════════════
# Create Datasets & DataLoaders
# ═══════════════════════════════════════════════════════════════════════════════
train_ds = LaTeXOCRDataset(train_data, tokenizer, CONFIG["img_size"], CONFIG["max_latex_len"])
val_ds = LaTeXOCRDataset(val_data, tokenizer, CONFIG["img_size"], CONFIG["max_latex_len"])

train_loader = DataLoader(train_ds, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=CONFIG["batch_size"]*2, shuffle=False, num_workers=2, pin_memory=True)

CONFIG["vocab_size"] = len(tokenizer)
print(f"\n📊 Train: {len(train_ds)} | Val: {len(val_ds)} | Vocab: {len(tokenizer)}")

# CELL 6: Training System

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 6: Training System with Real-time Exact Match
# ═══════════════════════════════════════════════════════════════════════════════

class Trainer:
    def __init__(self, model, train_loader, val_loader, cfg, tokenizer):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.cfg = cfg
        self.tokenizer = tokenizer
        
        # Optimizer
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=cfg["learning_rate"], 
                                           weight_decay=cfg["weight_decay"], betas=(0.9, 0.98))
        
        # Scheduler
        total_steps = cfg["num_epochs"] * len(train_loader)
        warmup = int(total_steps * cfg["warmup_ratio"])
        
        def lr_lambda(s):
            if s < warmup:
                return s / max(1, warmup)
            else:
                progress = (s - warmup) / max(1, total_steps - warmup)
                return 0.5 * (1 + math.cos(math.pi * progress))
        
        self.scheduler = torch.optim.lr_scheduler.LambdaLR(self.optimizer, lr_lambda)
        self.scaler = GradScaler('cuda') if cfg["use_amp"] else None
        self.best_acc = 0
        self.demo_sample = None
    
    def train_step(self, batch):
        batch = {k: v.to(device) for k, v in batch.items()}
        carry = self.model.initial_carry(batch)
        
        active_mask = torch.ones(batch["image"].shape[0], dtype=torch.bool, device=device)
        total_loss = 0.0
        valid_steps = 0
        final_preds = None
        
        self.optimizer.zero_grad()
        
        for step_idx in range(self.cfg["halt_max_steps"]):
            with autocast('cuda', enabled=self.cfg["use_amp"]):
                carry, out = self.model(carry, batch)
                target = carry.data["target"]
                
                seq_logits = out["seq"]
                ce_loss = F.cross_entropy(
                    seq_logits[:, :-1].reshape(-1, self.cfg["vocab_size"]),
                    target[:, 1:].reshape(-1),
                    ignore_index=self.tokenizer.PAD,
                    reduction='none'
                ).view(batch["image"].shape[0], -1).mean(dim=1)
                
                preds = seq_logits[:, :-1].argmax(-1)
                target_shifted = target[:, 1:]
                mask = target_shifted != self.tokenizer.PAD
                correct = ((preds == target_shifted) & mask).float().sum(-1) / mask.float().sum(-1).clamp(min=1)
                
                halt_target = (correct > self.cfg["halt_threshold"]).float()
                halt_loss = 0.05 * F.binary_cross_entropy_with_logits(
                    out["q_halt"], halt_target.detach(), reduction='none'
                )
                
                step_loss = ce_loss + halt_loss
                
                if active_mask.any():
                    loss_for_step = (step_loss * active_mask).sum() / (active_mask.sum() + 1e-6)
                    total_loss += loss_for_step
                    valid_steps += 1
                
                if final_preds is None:
                    final_preds = preds.clone()
                else:
                    final_preds = torch.where(active_mask.unsqueeze(-1), preds, final_preds)
                
                active_mask = active_mask & (~carry.halted)
            
            if (~active_mask).all():
                break
        
        if valid_steps > 0:
            total_loss = total_loss / valid_steps
            
        if self.scaler:
            self.scaler.scale(total_loss).backward()
            self.scaler.unscale_(self.optimizer)
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.scaler.step(self.optimizer)
            self.scaler.update()
        else:
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()
        self.scheduler.step()
        
        # 计算 token accuracy 和 exact match
        target = batch["target"][:, 1:]
        mask = target != self.tokenizer.PAD
        acc = ((final_preds == target) & mask).float().sum() / mask.float().sum()
        exact_match = ((final_preds == target) | ~mask).all(dim=1).float().mean()
        avg_steps = carry.steps.float().mean().item()
        
        return total_loss.item(), acc.item(), exact_match.item(), avg_steps
    
    @torch.no_grad()
    def evaluate(self):
        self.model.eval()
        total_correct, total_tokens, total_steps, total_samples = 0, 0, 0, 0
        exact_match = 0
        
        for batch in tqdm(self.val_loader, desc="Eval", leave=False):
            batch = {k: v.to(device) for k, v in batch.items()}
            carry = self.model.initial_carry(batch)
            
            active_mask = torch.ones(batch["image"].shape[0], dtype=torch.bool, device=device)
            final_preds = None
            
            for _ in range(self.cfg["halt_max_steps"]):
                carry, out = self.model(carry, batch)
                preds = out["seq"][:, :-1].argmax(-1)
                
                if final_preds is None:
                    final_preds = preds.clone()
                else:
                    final_preds = torch.where(active_mask.unsqueeze(-1), preds, final_preds)
                
                active_mask = active_mask & (~carry.halted)
                if (~active_mask).all():
                    break
            
            target = carry.data["target"][:, 1:]
            mask = target != self.tokenizer.PAD
            
            total_correct += ((final_preds == target) & mask).float().sum().item()
            total_tokens += mask.float().sum().item()
            total_steps += carry.steps.float().sum().item()
            total_samples += batch["image"].shape[0]
            
            seq_correct = ((final_preds == target) | ~mask).all(dim=1)
            exact_match += seq_correct.sum().item()
        
        self.model.train()
        token_acc = total_correct / total_tokens
        exact_acc = exact_match / total_samples
        avg_steps = total_steps / total_samples
        return token_acc, exact_acc, avg_steps
    
    @torch.no_grad()
    def predict_example(self, sample_idx=0):
        """预测一个样本并显示结果"""
        self.model.eval()
        
        if self.demo_sample is None:
            batch = next(iter(self.val_loader))
            self.demo_sample = {
                "image": batch["image"][sample_idx:sample_idx+1].to(device),
                "target": batch["target"][sample_idx:sample_idx+1].to(device)
            }
        
        sample = self.demo_sample
        carry = self.model.initial_carry(sample)
        for _ in range(self.cfg["halt_max_steps"]):
            carry, out = self.model(carry, sample)
            if carry.halted.all():
                break
        
        pred_ids = out["seq"][0, :-1].argmax(-1).cpu()
        target_ids = sample["target"][0, 1:].cpu()
        pred_text = self.tokenizer.decode(pred_ids)
        true_text = self.tokenizer.decode(target_ids)
        
        mask = target_ids != self.tokenizer.PAD
        token_acc = ((pred_ids == target_ids) & mask).float().sum() / mask.float().sum()
        exact_match = (pred_text == true_text)
        steps_used = carry.steps[0].item()
        
        self.model.train()
        
        return {
            "prediction": pred_text,
            "ground_truth": true_text,
            "token_accuracy": token_acc.item(),
            "exact_match": exact_match,
            "steps": steps_used,
            "image": sample["image"][0].cpu()
        }
    
    def train(self):
        # WANDB初始化
        wandb.login(key=self.cfg["wandb_api_key"])
        run = wandb.init(
            project=self.cfg["wandb_project"], 
            config=self.cfg,
            name=f"TRM-OCR-AR-H{self.cfg['H_cycles']}-L{self.cfg['L_cycles']}",
            settings=wandb.Settings(_disable_stats=False, _disable_meta=False)
        )
        print(f"\n🚀 Training (Autoregressive) | WANDB: {run.url}")
        
        step = 0
        for epoch in range(self.cfg["num_epochs"]):
            pbar = tqdm(self.train_loader, desc=f"Epoch {epoch+1}")
            
            for batch_idx, batch in enumerate(pbar):
                loss, acc, exact_match, avg_steps = self.train_step(batch)
                step += 1
                
                pbar.set_postfix(
                    loss=f"{loss:.4f}", 
                    acc=f"{acc:.3f}",
                    exact=f"{exact_match:.3f}",
                    steps=f"{avg_steps:.1f}",
                    lr=f"{self.scheduler.get_last_lr()[0]:.2e}"
                )
                
                if step % 10 == 0:
                    wandb.log({
                        "train/loss": loss, 
                        "train/token_acc": acc,
                        "train/exact_match": exact_match,
                        "train/steps": avg_steps, 
                        "train/lr": self.scheduler.get_last_lr()[0],
                        "train/epoch": epoch + (batch_idx / len(self.train_loader))
                    }, step=step)
            
            print(f"\n{'='*70}")
            print(f"📊 Epoch {epoch+1}/{self.cfg['num_epochs']} completed")
            print(f"{'='*70}")
            
            token_acc, exact_acc, val_steps = self.evaluate()
            print(f"📈 Validation Results:")
            print(f"   Token Acc: {token_acc:.4f} | Exact Match: {exact_acc:.4f} | Steps: {val_steps:.2f}")
            
            wandb.log({
                "val/token_acc": token_acc, 
                "val/exact_match": exact_acc, 
                "val/steps": val_steps, 
                "epoch": epoch + 1
            }, step=step)
            
            checkpoint_path = f"checkpoint_epoch_{epoch+1}.pt"
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'token_acc': token_acc,
                'exact_acc': exact_acc,
            }, checkpoint_path)
            print(f"💾 Saved checkpoint: {checkpoint_path}")
            
            print(f"\n🔍 Example Prediction:")
            result = self.predict_example()
            print(f"   Steps used: {result['steps']}")
            print(f"   Token Acc:  {result['token_accuracy']:.4f}")
            print(f"   Match: {'✅ EXACT' if result['exact_match'] else '❌ WRONG'}")
            print(f"\n   📝 Prediction:")
            print(f"      {result['prediction'][:100]}{'...' if len(result['prediction']) > 100 else ''}")
            print(f"\n   ✓ Ground Truth:")
            print(f"      {result['ground_truth'][:100]}{'...' if len(result['ground_truth']) > 100 else ''}")
            
            wandb.log({
                "example/prediction": result['prediction'],
                "example/ground_truth": result['ground_truth'],
                "example/token_acc": result['token_accuracy'],
                "example/exact_match": 1.0 if result['exact_match'] else 0.0,
                "example/steps": result['steps']
            }, step=step)
            
            if exact_acc > self.best_acc:
                self.best_acc = exact_acc
                torch.save(self.model.state_dict(), "best_model.pt")
                print(f"\n  🏆 New best model! Exact Match: {exact_acc:.4f}")
                wandb.run.summary["best_exact_match"] = exact_acc
                wandb.run.summary["best_token_acc"] = token_acc
            
            print(f"{'='*70}\n")
        
        wandb.finish()
        print(f"\n✅ Training Complete! Best Exact Match: {self.best_acc:.4f}")
        return self.best_acc

print("✅ Trainer (with Real-time Exact Match) defined")

# CELL 7: Create Model & Train

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 7: Create Model & Train
# ═══════════════════════════════════════════════════════════════════════════════

# Create model
model = TRMOCR(CONFIG, CONFIG["vocab_size"])
n_params = sum(p.numel() for p in model.parameters())
print(f"📊 Model: {n_params:,} params ({n_params*4/1024/1024:.1f} MB)")

# Create trainer
trainer = Trainer(model, train_loader, val_loader, CONFIG, tokenizer)

# Train!
best_acc = trainer.train()

# CELL 8: Test & Visualize

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 8: Test & Visualize
# ═══════════════════════════════════════════════════════════════════════════════

# Load best model
if os.path.exists("best_model.pt"):
    model.load_state_dict(torch.load("best_model.pt"))
    print("✅ Loaded best model")
model.eval()

# Test on sample batch
sample = next(iter(val_loader))
sample = {k: v.to(device) for k, v in sample.items()}

with torch.no_grad():
    carry = model.initial_carry(sample)
    active_mask = torch.ones(sample["image"].shape[0], dtype=torch.bool, device=device)
    final_preds = None
    
    print(f"\n🤔 Thinking Process:")
    for step in range(CONFIG["halt_max_steps"]):
        carry, out = model(carry, sample)
        preds = out["seq"].argmax(-1)
        
        if final_preds is None:
            final_preds = preds.clone()
        else:
            final_preds = torch.where(active_mask.unsqueeze(-1), preds, final_preds)
        
        just_halted = active_mask & carry.halted
        if just_halted.any():
            print(f"   Step {step+1}: {just_halted.sum().item()} samples halted.")
        
        active_mask = active_mask & (~carry.halted)
        if (~active_mask).all():
            break

target = carry.data["target"]
mask = target != tokenizer.PAD
token_acc = ((final_preds == target) & mask).float().sum() / mask.float().sum()
exact_match = ((final_preds == target) | ~mask).all(dim=1).float().mean()
avg_steps = carry.steps.float().mean().item()

print(f"\n🎯 Test Results:")
print(f"   Token Accuracy: {token_acc:.4f}")
print(f"   Exact Match: {exact_match:.4f}")
print(f"   Avg Steps: {avg_steps:.2f}")

print(f"\n📋 Sample Predictions:")
for i in range(min(5, len(final_preds))):
    pred_text = tokenizer.decode(final_preds[i])
    true_text = tokenizer.decode(target[i])
    step_count = carry.steps[i].item()
    match = '✓' if pred_text == true_text else '✗'
    
    print(f"\n   [{step_count:2d} steps] {match}")
    print(f"   Pred: {pred_text[:60]}{'...' if len(pred_text) > 60 else ''}")
    print(f"   True: {true_text[:60]}{'...' if len(true_text) > 60 else ''}")

# Visualize one
try:
    import matplotlib.pyplot as plt
    idx = 0
    img = sample["image"][idx].permute(1, 2, 0).cpu().numpy()
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img = np.clip(img, 0, 1)
    
    plt.figure(figsize=(8, 4))
    plt.imshow(img)
    plt.title(f"Pred: {tokenizer.decode(final_preds[idx])[:50]}...", fontsize=10)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Visualization skipped: {e}")